In [ ]:
import numpy as np
from astropy.visualization import ZScaleInterval, PercentileInterval
from ipywidgets import interact

import bokeh
from bokeh.palettes import viridis, Category10
from bokeh.plotting import figure, output_file, show, output_notebook, save
from bokeh.models import GlyphRenderer, LinearColorMapper
from bokeh.models import LinearAxis, Range1d
from bokeh.io import push_notebook
from bokeh.layouts import gridplot, layout, row, column

#from numba import jit, njit

bokeh.__version__

In [ ]:
output_notebook()

In [ ]:
%mpdaf

In [ ]:
from astropy.visualization import ZScaleInterval, PercentileInterval, MinMaxInterval
from bokeh.models import LogColorMapper, LogTicker, ColorBar, LinearColorMapper

def plot_image(im, size=(350, 350), title=None, colorbar=True, palette="Viridis256", scale='minmax', axis=True, x_range=None, y_range=None):
    if scale == 'zscale':
        interval = ZScaleInterval()
    elif scale == 'percentile':
        interval = PercentileInterval(99)
    elif scale == 'minmax':
        interval = MinMaxInterval()

    vmin, vmax = interval.get_limits(im.data)
    color_mapper = LinearColorMapper(palette=palette, low=vmin, high=vmax)
    if x_range is None:
        x_range = (0, im.shape[0])
    if y_range is None:
        y_range = (0, im.shape[1])

    p = figure(plot_width=size[0], plot_height=size[1], 
               x_range=x_range, y_range=y_range, title=title)
    p.image(image=[im.data.filled()], x=[0], y=[0], dw=[im.shape[0]], dh=[im.shape[1]], 
            color_mapper=color_mapper)
    p.grid.visible = False
    p.axis.visible = axis

    if colorbar:
        color_bar = ColorBar(color_mapper=color_mapper, #ticker=LogTicker(),
                             label_standoff=12, border_line_color=None, location=(0,0))
        p.add_layout(color_bar, 'right')

    return p

In [ ]:
def images(s, size=(350, 350)):
    images = {
        'MUSE_WHITE': {'scale': 'zscale'},
        'MUSE_EXPMAP': {'scale': 'minmax', 'palette': 'Greys256', 'link': 'MUSE_WHITE'},
        'MASK_OBJ': {'scale': 'minmax', 'palette': 'Greys256', 'link': 'MUSE_WHITE'}, 
        'HST_F606W': {'scale': 'percentile'}, 
        'HST_F775W': {'scale': 'percentile', 'link': 'HST_F606W'}
    }
    for name, params in images.items():
        im = s.images[name]
        kw = dict(size=size, title=name, colorbar=False, axis=False,
                  scale=params['scale'], palette=params.get('palette', 'Viridis256'))
        if params.get('link') is not None:
            kw['x_range'] = images[params['link']]['plot'].x_range
            kw['y_range'] = images[params['link']]['plot'].y_range
            
        s2 = plot_image(im, **kw)

        if name == 'MUSE_WHITE':
            y, x = im.wcs.sky2pix((s.DEC, s.RA))[0]
            s2.circle(y, x, size=4, color='red', alpha=1)
        
        params['plot'] = s2

    return [im['plot'] for im in images.values()]

In [ ]:
def spectrum(s, size=(800, 350), axis_labels=True, lbrange=None, legend=True):
    s1 = figure(plot_width=size[0], plot_height=size[1])
                #title=f"Origin source #{s.ID}")

    snames = ['MUSE_TOT_SKYSUB', 'MUSE_PSF_SKYSUB', 'MUSE_WHITE_SKYSUB']
    palette = Category10[8]

    smin, smax = np.inf, -np.inf
    for i, sname in enumerate(snames):
        sp = s.spectra[sname]
        smin = min(smin, sp.data.min())
        smax = max(smax, sp.data.max())
        if lbrange:
            sp = sp.subspec(lbrange[0], lbrange[1])
        line = s1.line(sp.wave.coord(), sp.data, legend=sname.lstrip('MUSE_'), color=palette[i])
        if i > 0:
            line.visible = False

    s1.extra_y_ranges = {"var": Range1d(start=0, end=sp.var.max())}
    s1.y_range = Range1d(smin - 20, smax + 20)
    s1.line(sp.wave.coord(), sp.var, line_color='gray', line_alpha=0.6, y_range_name="var")
    #s1.add_layout(LinearAxis(y_range_name="var"), 'left')

    s1.legend.location = "top_left"
    s1.legend.visible = legend
    if axis_labels:
        s1.xaxis.axis_label = f'Wavelength ({sp.wave.unit})'
        s1.yaxis.axis_label = f'Flux ({sp.unit})'
    
    s1.legend.click_policy = "hide"
    
    return s1

In [ ]:
from bokeh.models.widgets import Div

def legend(s, size=(800, 200)):
    z = s.z[s.z['Z_DESC'] == 'MUSE'][0]['Z']
    div = Div(text=f"""<h4>Source #{s.ID}, ({s.RA:.6f}, {s.DEC:.6f}), z={z:.3f}</h4>""", width=size[0], height=size[1])
    return [div]

In [ ]:
from mpdaf.sdetect import get_emlines

def plot_lines(s, size=(250,250)):
    #z = s.z[s.z['Z_DESC'] == 'MUSE'][0]['Z']
    #lines = get_emlines(z=z, lbrange=sp.wave.get_range(), sel=1, table=True, ltype='em')
    #lines.add_index('LINE')
    palette = Category10[8]
    
    sp = s.spectra['MUSE_TOT_SKYSUB']
    s.lines.sort('FLUX_REF')
    lines = s.lines[-2:]

    figures = []
    for line in lines:
        #lbda = lines.loc[line]['LBDA_OBS']
        s1 = figure(plot_width=size[0], plot_height=size[1], title=line['LINE'])
        lbda = line['LBDA_OBS']
        subsp = sp.subspec(lbda-25, lbda+25)
        line = s1.line(subsp.wave.coord(), subsp.data, color=palette[0])
        figures.append(s1)
        
    return figures

In [ ]:
#s = Source.from_file('/home/simon/data/UDF/udf_origin_00223.fits')
#s = Source.from_file('/home/simon/data/UDF/udf10_c031_e021/udf_udf10_00002.fits')
s = Source.from_file('/home/simon/data/UDF/udf_udf10_00002.fits')

In [ ]:
#output_file('output.html')

if True:
    l = gridplot([
        legend(s, size=(1250, 20)),
        images(s, size=(250,250)), 
        [spectrum(s, size=(750, 250), axis_labels=False)] + plot_lines(s)
    ], sizing_mode='fixed')
else:
    l = layout([
        legend(s, size=(1250, 20)),
        images(s, size=(250,250)), 
        [spectrum(s, size=(750, 250), axis_labels=False)] + plot_lines(s)
    ], sizing_mode='stretch_both')

show(l)
#plot = show(p, notebook_handle=True)

In [ ]:
p = plot_image(s.images['MUSE_WHITE'], colorbar=False, size=(200, 200), axis=False)
p2 = plot_image(s.images['HST_F606W'], colorbar=False, size=(200, 200), axis=False)
show(gridplot([[p, p2]]))

In [ ]:
show(gridplot([        
    images(s, size=(200,200)), 
]))

In [ ]:
show(column(        
    row(images(s, size=(200,200))), 
    row(spectrum(s, size=(750, 250)), *plot_lines(s))
))